In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from gensim.models import doc2vec
# TaggedDocument는 해당 corpus의 id를 함께 넘겨주는 것을 말함
from gensim.models.doc2vec import TaggedDocument
import time
import tensorflow_datasets as tfds
import tensorflow as tf

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1. colab : !pip install konlpy 설치해야 함
1. https://cholol.tistory.com/466
1. https://github.com/tkdlek11112/faq_chatbot_learning/blob/master/DAY1/FAQ_CHATBOT_DAY1.ipynb


In [2]:
df = pd.read_csv('ChatBotData.csv')
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
df[['Q','A']]

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.
...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.
11820,흑기사 해주는 짝남.,설렜겠어요.
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.


In [4]:
#형태소 분석
import jpype
from konlpy.tag import Okt

In [17]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()

def tokenizer_okt(doc):
    jpype.attachThreadToJVM()
    token_doc = ['/'.join(word) for word in okt.pos(doc) if not word in stopwords ]
    return token_doc

In [18]:
tokenizer_okt(df['Q'][0])

['12시/Number', '땡/Noun', '!/Punctuation']

In [20]:
# 구성 : [ index, 형태소분석(질문), 답 ] 
pre_df = []
for i in range(len(df)):
    pre_df.append( [i,tokenizer_okt(df['Q'][i]),df['A'][i] ])

pre_df[:5]    

[[0, ['12시/Number', '땡/Noun', '!/Punctuation'], '하루가 또 가네요.'],
 [1, ['1/Number', '지망/Noun', '학교/Noun', '떨어졌어/Verb'], '위로해 드립니다.'],
 [2,
  ['3/Number', '박/Noun', '4일/Number', '놀러/Verb', '가고/Verb', '싶다/Verb'],
  '여행은 언제나 좋죠.'],
 [3,
  ['3/Number',
   '박/Noun',
   '4일/Number',
   '정도/Noun',
   '놀러/Verb',
   '가고/Verb',
   '싶다/Verb'],
  '여행은 언제나 좋죠.'],
 [4, ['PPL/Alpha', '심하네/Adjective'], '눈살이 찌푸려지죠.']]

In [21]:
# Doc2Vec에서 사용하는 태그문서형으로 변경
tagged_faqs = [TaggedDocument(que, [idx]) for idx, que,ans in pre_df]

In [22]:
tagged_faqs[:5]

[TaggedDocument(words=['12시/Number', '땡/Noun', '!/Punctuation'], tags=[0]),
 TaggedDocument(words=['1/Number', '지망/Noun', '학교/Noun', '떨어졌어/Verb'], tags=[1]),
 TaggedDocument(words=['3/Number', '박/Noun', '4일/Number', '놀러/Verb', '가고/Verb', '싶다/Verb'], tags=[2]),
 TaggedDocument(words=['3/Number', '박/Noun', '4일/Number', '정도/Noun', '놀러/Verb', '가고/Verb', '싶다/Verb'], tags=[3]),
 TaggedDocument(words=['PPL/Alpha', '심하네/Adjective'], tags=[4])]

In [24]:
# make model
import multiprocessing
cores = multiprocessing.cpu_count()
d2v_faqs = doc2vec.Doc2Vec(
                            vector_size=100,
                            alpha=0.025,
                            min_alpha=0.025,
                            hs=1,
                            negative=0,
                            dm=0,
                            window=3,
                            dbow_words = 1,
                            min_count = 1,
                            workers = cores,
                            seed=0,
                            epochs=100
                            )
d2v_faqs.build_vocab(tagged_faqs)


In [1]:
# d2v_faqs.vocabulary.sample

In [25]:
# 검색타입 변경
test_string = "1지망 학교 떨어졌어"
# 검색할 데이터 형태소분석
tokened_test_string = tokenizer_okt(test_string)
tokened_test_string

['1/Number', '지망/Noun', '학교/Noun', '떨어졌어/Verb']

In [26]:
topn = 5
# 검색할 데이터 벡터화
test_vector = d2v_faqs.infer_vector(tokened_test_string)
# 가장 근접한 5개 list 돌려줌.
result = d2v_faqs.docvecs.most_similar([test_vector], topn=topn)
print(result)

for i in range(topn):
    print("{}위. {}, {} {}".format(i+1, result[i][1], result[i][0],df['Q'][result[i][0]] ))

[(8725, 0.37584811449050903), (8337, 0.33695921301841736), (8276, 0.3326198160648346), (2779, 0.33130156993865967), (7508, 0.32867804169654846)]
1위. 0.37584811449050903, 8725 헤이진지 이제 딱 한달
2위. 0.33695921301841736, 8337 출근 준비하다 또 울었네
3위. 0.3326198160648346, 8276 찬 사람은 후폭풍 없나?
4위. 0.33130156993865967, 2779 신혼여행 한달쯤 가고 싶다
5위. 0.32867804169654846, 7508 이별은 냉정할수록 좋은 것 같아


C:\Users\jmpkorea00\AppData\Local\Temp\ipykernel_3052\1373282670.py:5: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  result = d2v_faqs.docvecs.most_similar([test_vector], topn=topn)


In [27]:
# 성능 측정
raten = 1
found = 0
for i in range(len(df)):
    tstr = df['Q'][i]
    ttok = tokenizer_okt(tstr)
    tvec = d2v_faqs.infer_vector(ttok)
    re = d2v_faqs.docvecs.most_similar([tvec], topn = raten)
    for j in range(raten):
        if i == re[j][0]: found = found + 1

print("정확도 = {} % ({}/{} )  ".format(found/len(df),found, len(df)))

C:\Users\jmpkorea00\AppData\Local\Temp\ipykernel_3052\1411992680.py:8: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  re = d2v_faqs.docvecs.most_similar([tvec], topn = raten)


정확도 = 0.0001691618032648228 % (2/11823 )  
